# Midterm Dataset Verification + Baseline ML Pipeline
## Dataset: HR Analytics - Job Change of Data Scientists (Kaggle)

Dataset URL:
https://www.kaggle.com/datasets/arashnic/hr-analytics-job-change-of-data-scientists

Goals:
1) Verify dataset meets requirements:
   - >= 2000 rows
   - Mix of categorical and numeric columns
   - Missing values present (ideally meaningful)
2) Correct ML workflow:
   - Split dataset into training 70% and testing 30% FIRST
   - Perform EDA only on training data
   - Keep testing data untouched until evaluation
3) Data preparation:
   - Fill NaN/Null values
   - Scaling numeric features
   - Normalization (log transform for skewed numeric columns)
   - One-hot encoding / encoding categorical columns
   - Feature engineering
4) Train baseline model and evaluate
5) Export cleaned train/test datasets

In [10]:
# Cell 2 - Imports
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_auc_score

In [11]:
# Cell 3 - Load Dataset (CSV)
# DATA_PATH = "/content/aug_train.csv"  # update if needed
# df = pd.read_csv(DATA_PATH)
# df.head()
DATA_URL = "https://raw.githubusercontent.com/iamhuongthn/ITAI-1371_Mid_Term_EDA_Group_ML_19179_3/master/data/raw/aug_train.csv"
print("Loading dataset from:", DATA_URL)

df = pd.read_csv(DATA_URL)
print("Shape:", df.shape)
df.head()

Loading dataset from: https://raw.githubusercontent.com/iamhuongthn/ITAI-1371_Mid_Term_EDA_Group_ML_19179_3/master/data/raw/aug_train.csv
Shape: (19158, 14)


,enrollee_id,city,city_development_index,gender,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,target
0,8949,city_103,0.920,Male,Has relevent experience,no_enrollment,Graduate,STEM,>20,NaN,NaN,1,36,1.0
1,29725,city_40,0.776,Male,No relevent experience,no_enrollment,Graduate,STEM,15,50-99,Pvt Ltd,>4,47,0.0
2,11561,city_21,0.624,NaN,No relevent experience,Full time course,Graduate,STEM,5,NaN,NaN,never,83,0.0
3,33241,city_115,0.789,NaN,No relevent experience,NaN,Graduate,Business Degree,<1,NaN,Pvt Ltd,never,52,1.0
4,666,city_162,0.767,Male,Has relevent experience,no_enrollment,Masters,STEM,>20,50-99,Funded Startup,4,8,0.0


In [12]:
# Cell 4 - Requirement Check: Rows, Types, Missing
print("Shape:", df.shape)
assert df.shape[0] >= 2000, "❌ Dataset has fewer than 2000 rows"

print("\nData types summary:")
print(df.dtypes.value_counts())

num_cols = df.select_dtypes(include=["number"]).columns.tolist()
cat_cols = df.select_dtypes(include=["object", "category", "bool"]).columns.tolist()

print(f"\nNumeric columns: {len(num_cols)}")
print(f"Categorical columns: {len(cat_cols)}")

assert len(num_cols) > 0, "❌ No numeric columns found"
assert len(cat_cols) > 0, "❌ No categorical columns found"

missing_pct = df.isnull().mean() * 100
print("\nTop missing columns (%):")
print(missing_pct.sort_values(ascending=False).head(15))

overall_missing = df.isnull().values.mean() * 100
print(f"\nOverall missing (entire table): {overall_missing:.2f}%")

print("\n✅ Passed: >=2000 rows + mixed numeric/categorical.")
print("✅ Missing values check completed (see % above).")

Shape: (19158, 14)

Data types summary:
object     10
int64       2
float64     2
Name: count, dtype: int64

Numeric columns: 4
Categorical columns: 10

Top missing columns (%):
company_type              32.049274
company_size              30.994885
gender                    23.530640
major_discipline          14.683161
education_level            2.401086
last_new_job               2.207955
enrolled_university        2.014824
experience                 0.339284
enrollee_id                0.000000
city                       0.000000
relevent_experience        0.000000
city_development_index     0.000000
training_hours             0.000000
target                     0.000000
dtype: float64

Overall missing (entire table): 7.73%

✅ Passed: >=2000 rows + mixed numeric/categorical.
✅ Missing values check completed (see % above).


In [13]:
# Cell 5 - Target Column Check
TARGET_COL = "target"
assert TARGET_COL in df.columns, f"❌ Target '{TARGET_COL}' not found. Columns: {df.columns.tolist()}"

print(df[TARGET_COL].value_counts())
print("\nTarget distribution:")
print(df[TARGET_COL].value_counts(normalize=True))

target
0.0    14381
1.0     4777
Name: count, dtype: int64

Target distribution:
target
0.0    0.750652
1.0    0.249348
Name: proportion, dtype: float64


In [14]:
# Cell 6 - Split FIRST (70/30)
X = df.drop(columns=[TARGET_COL])
y = df[TARGET_COL]

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.30,
    random_state=42,
    stratify=y if y.nunique() == 2 else None
)

print("Train shape:", X_train.shape, "Test shape:", X_test.shape)
print("\nTarget distribution (train):")
print(y_train.value_counts(normalize=True))

Train shape: (13410, 13) Test shape: (5748, 13)

Target distribution (train):
target
0.0    0.750634
1.0    0.249366
Name: proportion, dtype: float64


In [15]:
# Cell 7 - EDA Only on Training Set
train_df = X_train.copy()
train_df[TARGET_COL] = y_train

train_df.info()

print("\nMissing % (TRAINING ONLY):")
train_missing_pct = train_df.isnull().mean() * 100
print(train_missing_pct.sort_values(ascending=False).head(15))

print("\nNumeric summary (TRAINING ONLY):")
display(train_df.select_dtypes(include=["number"]).describe().T)

<class 'pandas.core.frame.DataFrame'>
Index: 13410 entries, 9719 to 3085
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   enrollee_id             13410 non-null  int64  
 1   city                    13410 non-null  object 
 2   city_development_index  13410 non-null  float64
 3   gender                  10287 non-null  object 
 4   relevent_experience     13410 non-null  object 
 5   enrolled_university     13145 non-null  object 
 6   education_level         13093 non-null  object 
 7   major_discipline        11420 non-null  object 
 8   experience              13364 non-null  object 
 9   company_size            9297 non-null   object 
 10  company_type            9130 non-null   object 
 11  last_new_job            13118 non-null  object 
 12  training_hours          13410 non-null  int64  
 13  target                  13410 non-null  float64
dtypes: float64(2), int64(2), object(10)
memor

,count,mean,std,min,25%,50%,75%,max
enrollee_id,13410.0,16896.888292,9585.732018,1.000,8581.25,16950.00,25152.75,33377.000
city_development_index,13410.0,0.829212,0.123273,0.448,0.74,0.91,0.92,0.949
training_hours,13410.0,65.330798,60.048712,1.000,23.00,47.00,88.00,336.000
target,13410.0,0.249366,0.432662,0.000,0.00,0.00,0.00,1.000


In [16]:
# View a few top categorical columns
cat_cols_train = train_df.select_dtypes(include=["object", "category", "bool"]).columns.tolist()
for c in cat_cols_train[:8]:
    print(f"\n{c} value counts (top 10):")
    print(train_df[c].value_counts(dropna=False).head(10))


city value counts (top 10):
city
city_103    3056
city_21     1877
city_16     1079
city_114     922
city_160     588
city_136     408
city_67      303
city_75      220
city_102     214
city_104     214
Name: count, dtype: int64

gender value counts (top 10):
gender
Male      9309
NaN       3123
Female     841
Other      137
Name: count, dtype: int64

relevent_experience value counts (top 10):
relevent_experience
Has relevent experience    9636
No relevent experience     3774
Name: count, dtype: int64

enrolled_university value counts (top 10):
enrolled_university
no_enrollment       9679
Full time course    2612
Part time course     854
NaN                  265
Name: count, dtype: int64

education_level value counts (top 10):
education_level
Graduate          8095
Masters           3036
High School       1428
NaN                317
Phd                313
Primary School     221
Name: count, dtype: int64

major_discipline value counts (top 10):
major_discipline
STEM               10132

In [17]:
# Cell 8 - Feature Engineering
X_train_fe = X_train.copy()
X_test_fe = X_test.copy()

def parse_experience(x):
    if pd.isna(x):
        return np.nan
    s = str(x).strip()
    if s == ">20":
        return 21
    if s == "<1":
        return 0
    # sometimes values are 'never' (for last_new_job, not experience)
    try:
        return float(s)
    except:
        return np.nan

def parse_last_new_job(x):
    if pd.isna(x):
        return np.nan
    s = str(x).strip()
    if s == "never":
        return 0
    if s == ">4":
        return 5
    try:
        return float(s)
    except:
        return np.nan

def add_features(d: pd.DataFrame) -> pd.DataFrame:
    d = d.copy()

    # Convert string-coded numeric fields
    if "experience" in d.columns:
        d["experience_num"] = d["experience"].apply(parse_experience)
    if "last_new_job" in d.columns:
        d["last_new_job_num"] = d["last_new_job"].apply(parse_last_new_job)

    # Example ratio feature if both exist
    if "training_hours" in d.columns and "experience_num" in d.columns:
        d["training_per_exp"] = d["training_hours"] / (d["experience_num"].replace(0, np.nan))

    # Seniority bucket (categorical engineered)
    if "experience_num" in d.columns:
        d["exp_bucket"] = pd.cut(
            d["experience_num"],
            bins=[-1, 0, 2, 5, 10, 20, np.inf],
            labels=["0", "1-2", "3-5", "6-10", "11-20", "20+"]
        ).astype(str)

    # Drop pure identifier column(s) if present
    for id_col in ["enrollee_id"]:
        if id_col in d.columns:
            d = d.drop(columns=[id_col])

    return d

X_train_fe = add_features(X_train_fe)
X_test_fe = add_features(X_test_fe)

new_cols = [c for c in X_train_fe.columns if c not in X_train.columns]
print("✅ Engineered columns:", new_cols)
print("Train shape after FE:", X_train_fe.shape)

✅ Engineered columns: ['experience_num', 'last_new_job_num', 'training_per_exp', 'exp_bucket']
Train shape after FE: (13410, 16)


In [18]:
# Cell 9 - Normalization Demo (log1p on skewed numeric)
X_train_norm = X_train_fe.copy()
X_test_norm = X_test_fe.copy()

for col in ["training_hours"]:
    if col in X_train_norm.columns:
        X_train_norm[f"{col}_log1p"] = np.log1p(X_train_norm[col].clip(lower=0))
        X_test_norm[f"{col}_log1p"] = np.log1p(X_test_norm[col].clip(lower=0))

print("✅ Normalized columns added:", [c for c in X_train_norm.columns if c.endswith("_log1p")])

✅ Normalized columns added: ['training_hours_log1p']


In [19]:
# Cell 10 - Build Preprocessing Pipeline (Impute + OneHot + Scale)
numeric_features = X_train_norm.select_dtypes(include=["number"]).columns.tolist()
categorical_features = X_train_norm.select_dtypes(include=["object", "category", "bool"]).columns.tolist()

print("Numeric features:", len(numeric_features))
print("Categorical features:", len(categorical_features))

numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore"))
])

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features)
    ],
    remainder="drop"
)

Numeric features: 6
Categorical features: 11


In [20]:
# Cell 11 - Baseline Model + Evaluation
model = Pipeline(steps=[
    ("preprocess", preprocessor),
    ("clf", LogisticRegression(max_iter=2000))
])

model.fit(X_train_norm, y_train)

pred = model.predict(X_test_norm)
proba = model.predict_proba(X_test_norm)[:, 1] if y.nunique() == 2 else None

print("Accuracy:", accuracy_score(y_test, pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, pred))
print("\nClassification Report:\n", classification_report(y_test, pred))

if proba is not None:
    print("ROC AUC:", roc_auc_score(y_test, proba))

Accuracy: 0.7748782185107863

Confusion Matrix:
 [[3980  335]
 [ 959  474]]

Classification Report:
               precision    recall  f1-score   support

         0.0       0.81      0.92      0.86      4315
         1.0       0.59      0.33      0.42      1433

    accuracy                           0.77      5748
   macro avg       0.70      0.63      0.64      5748
weighted avg       0.75      0.77      0.75      5748

ROC AUC: 0.7874361091277526


In [21]:
# Cell 12 - Export Cleaned Train/Test (Model-Ready CSV)
X_train_clean = model.named_steps["preprocess"].transform(X_train_norm)
X_test_clean = model.named_steps["preprocess"].transform(X_test_norm)

ohe = model.named_steps["preprocess"].named_transformers_["cat"].named_steps["onehot"]
cat_feature_names = ohe.get_feature_names_out(categorical_features)
feature_names = np.concatenate([numeric_features, cat_feature_names])

X_train_arr = X_train_clean.toarray() if hasattr(X_train_clean, "toarray") else X_train_clean
X_test_arr = X_test_clean.toarray() if hasattr(X_test_clean, "toarray") else X_test_clean

train_clean_df = pd.DataFrame(X_train_arr, columns=feature_names)
test_clean_df = pd.DataFrame(X_test_arr, columns=feature_names)

train_clean_df[TARGET_COL] = y_train.reset_index(drop=True).values
test_clean_df[TARGET_COL] = y_test.reset_index(drop=True).values

train_clean_df.to_csv("hr_jobchange_train_clean.csv", index=False)
test_clean_df.to_csv("hr_jobchange_test_clean.csv", index=False)

print("✅ Saved: hr_jobchange_train_clean.csv and hr_jobchange_test_clean.csv")
train_clean_df.head()

✅ Saved: hr_jobchange_train_clean.csv and hr_jobchange_test_clean.csv


,city_development_index,training_hours,experience_num,last_new_job_num,training_per_exp,training_hours_log1p,city_city_1,city_city_10,city_city_100,city_city_101,...,last_new_job_>4,last_new_job_never,exp_bucket_0,exp_bucket_1-2,exp_bucket_11-20,exp_bucket_20+,exp_bucket_3-5,exp_bucket_6-10,exp_bucket_nan,target
0,-0.326213,1.310139,-1.051250,-1.187630,1.940392,1.246571,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
1,-1.194241,-0.888159,-0.903768,-0.587165,-0.440354,-1.308495,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,-1.064442,-0.255316,-1.198733,-1.187630,0.723566,0.139586,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.736512,-0.788236,0.571058,1.814692,-0.531049,-0.906460,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,0.736512,-0.088778,0.866023,1.814692,-0.400675,0.329272,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
